In [ ]:
# Quantum Simulation Solvers Demo

"""
Demonstration of the refactored quantum simulation solvers.
This notebook shows how to use different time evolution algorithms:
- Exact ODE solver
- VQA compilation solver  
- Trotter decomposition (1st & 2nd order)
- Magnus expansion (2nd order)
"""

import numpy as np
import time
from quantum_simulation import (
    create_quantum_system, 
    create_exact_solver,
    create_vqa_solver, 
    create_trotter_solver,
    create_magnus_solver,
    SimulationConfig
)

print("=== Quantum Simulation Solvers Demo ===")

# Create shorter simulation for demo
demo_config = SimulationConfig(
    time_end=10.0,      # Shorter time for demo
    num_time_points=5,  # Fewer points for demo
    max_optimization_steps=50,  # Fewer VQA steps for demo
    num_layers=2        # Fewer layers for demo
)

print(f"Demo configuration:")
print(f"- Time range: {demo_config.time_start} to {demo_config.time_end}")
print(f"- Time points: {demo_config.num_time_points}")
print(f"- VQA layers: {demo_config.num_layers}")
print(f"- VQA max steps: {demo_config.max_optimization_steps}")


In [ ]:
# Step 1: Setup quantum system

print("\n=== Step 1: Setup Quantum System ===")

# Create quantum system
system = create_quantum_system(demo_config)

# Get ground state
ground_state, ground_energy = system.compute_ground_state_vqe()

# Get time points for evolution
time_points = demo_config.get_time_points()

print(f"System setup complete:")
print(f"- Ground energy: {ground_energy:.6f} hartree")
print(f"- System has {system.static_hamiltonian.num_qubits} qubits")
print(f"- Time points: {time_points}")
print(f"- Initial state norm: {np.linalg.norm(ground_state):.6f}")


In [ ]:
# Step 2: Test Exact ODE Solver

print("\n=== Step 2: Exact ODE Solver ===")

# Create exact solver
exact_solver = create_exact_solver(system, demo_config)

# Run evolution
start_time = time.time()
exact_states = exact_solver.evolve(time_points, ground_state)
exact_time = time.time() - start_time

print(f"Exact solver results:")
print(f"- Evolution time: {exact_time:.3f} seconds")
print(f"- Solver info: {exact_solver.get_solver_info()['results']}")

# Check state norms
for t, state in exact_states.items():
    norm = np.linalg.norm(state)
    print(f"- t={t:4.1f}: state norm = {norm:.6f}")

print("✅ Exact solver completed successfully!")


In [ ]:
# Step 3: Test Trotter Solvers

print("\n=== Step 3: Trotter Solvers ===")

# Test Trotter 1st order
print("Testing Trotter 1st order...")
trotter1_solver = create_trotter_solver(system, demo_config, order=1)

start_time = time.time()
trotter1_states = trotter1_solver.evolve(time_points, ground_state)
trotter1_time = time.time() - start_time

print(f"Trotter 1st order results:")
print(f"- Evolution time: {trotter1_time:.3f} seconds")
print(f"- Solver info: {trotter1_solver.get_solver_info()['results']}")

# Test Trotter 2nd order
print("\nTesting Trotter 2nd order...")
trotter2_solver = create_trotter_solver(system, demo_config, order=2)

start_time = time.time()
trotter2_states = trotter2_solver.evolve(time_points, ground_state)
trotter2_time = time.time() - start_time

print(f"Trotter 2nd order results:")
print(f"- Evolution time: {trotter2_time:.3f} seconds")
print(f"- Solver info: {trotter2_solver.get_solver_info()['results']}")

print("✅ Both Trotter solvers completed successfully!")


In [ ]:
# Step 4: Test Magnus Solver

print("\n=== Step 4: Magnus Solver ===")

# Create Magnus solver
magnus_solver = create_magnus_solver(system, demo_config)

# Run evolution
start_time = time.time()
magnus_states = magnus_solver.evolve(time_points, ground_state)
magnus_time = time.time() - start_time

print(f"Magnus 2nd order results:")
print(f"- Evolution time: {magnus_time:.3f} seconds")
print(f"- Solver info: {magnus_solver.get_solver_info()['results']}")

# Check state norms
for t, state in magnus_states.items():
    norm = np.linalg.norm(state)
    print(f"- t={t:4.1f}: state norm = {norm:.6f}")

print("✅ Magnus solver completed successfully!")


In [ ]:
# Step 5: Test VQA Compilation Solver (optional - takes longer)

# VQA solver is compute-intensive, so we'll test it separately
print("\n=== Step 5: VQA Compilation Solver (Optional) ===")
print("Warning: VQA solver is computationally intensive!")

test_vqa = input("Run VQA solver? (y/n): ").lower().strip() == 'y'

if test_vqa:
    print("Testing VQA compilation solver...")
    vqa_solver = create_vqa_solver(system, demo_config)
    
    start_time = time.time()
    vqa_states = vqa_solver.evolve(time_points, ground_state)
    vqa_time = time.time() - start_time
    
    print(f"VQA compilation results:")
    print(f"- Evolution time: {vqa_time:.3f} seconds")
    print(f"- Solver info: {vqa_solver.get_solver_info()['results']}")
    
    print("✅ VQA solver completed successfully!")
else:
    print("⏭️ Skipping VQA solver (recommended for demo)")
    vqa_states = None
    vqa_time = None


In [ ]:
# Step 6: Performance Comparison and Summary

print("\n=== Step 6: Performance Summary ===")

# Create performance comparison table
print("Solver Performance Comparison:")
print("=" * 60)
print(f"{'Solver':<20} {'Time (s)':<12} {'Status':<15} {'Notes'}")
print("-" * 60)

print(f"{'Exact ODE':<20} {exact_time:<12.3f} {'✅ Success':<15} Reference solution")
print(f"{'Trotter 1st':<20} {trotter1_time:<12.3f} {'✅ Success':<15} Fast approximation")
print(f"{'Trotter 2nd':<20} {trotter2_time:<12.3f} {'✅ Success':<15} Better accuracy")
print(f"{'Magnus 2nd':<20} {magnus_time:<12.3f} {'✅ Success':<15} High accuracy")

if test_vqa and vqa_time is not None:
    print(f"{'VQA Compilation':<20} {vqa_time:<12.3f} {'✅ Success':<15} Quantum circuit")
else:
    print(f"{'VQA Compilation':<20} {'--':<12} {'⏭️ Skipped':<15} Compute intensive")

print("=" * 60)

# Calculate some basic error metrics (using exact as reference)
print("\nAccuracy Analysis (vs Exact ODE):")
print("-" * 40)

def calculate_state_error(states1, states2, time_points):
    """Calculate average state fidelity error."""
    errors = []
    for t in time_points:
        if t in states1 and t in states2:
            fidelity = np.abs(np.vdot(states1[t], states2[t]))**2
            error = 1 - fidelity
            errors.append(error)
    return np.mean(errors) if errors else float('inf')

trotter1_error = calculate_state_error(exact_states, trotter1_states, time_points)
trotter2_error = calculate_state_error(exact_states, trotter2_states, time_points)
magnus_error = calculate_state_error(exact_states, magnus_states, time_points)

print(f"Trotter 1st order: Avg fidelity error = {trotter1_error:.2e}")
print(f"Trotter 2nd order: Avg fidelity error = {trotter2_error:.2e}")
print(f"Magnus 2nd order:  Avg fidelity error = {magnus_error:.2e}")

if test_vqa and vqa_states is not None:
    vqa_error = calculate_state_error(exact_states, vqa_states, time_points)
    print(f"VQA Compilation:   Avg fidelity error = {vqa_error:.2e}")

print("\n🎉 All solvers demo completed successfully!")
print("📊 The refactored code provides a clean, modular interface for quantum simulation!")
